In [1]:
import numpy as np
import netCDF4 as nc
import scipy as sc

In [2]:
!ls

bathy_meter.nc			      u_unrotated_wcvi_t_3D.npy
C:\nppdf32Log\debuglog.txt	      u_unrotated_wcvi_t.npy
coordinates.nc			      V_3D_wcvi_bottom.npy
Detiding.ipynb			      V_3D_wcvi_left.npy
grid_alignment.py		      V_3D_wcvi_right.npy
Grid_transformation.ipynb	      V_3D_wcvi_top.npy
JP_Grid.png			      vosaline_bottombc.npy
Making_IC_from_JP.ipynb		      vosaline_leftbc.npy
Making_lateral_boundaries.ipynb       vosaline_rightbc.npy
Plotting_data_over_3_locations.ipynb  vosaline_topbc.npy
__pycache__			      votemper_bottombc.npy
ssh_bottombc.npy		      votemper_leftbc.npy
ssh_leftbc.npy			      votemper_rightbc.npy
ssh_rightbc.npy			      votemper_topbc.npy
ssh_topbc.npy			      v_unrotated_wcvi_t_3D.npy
U_3D_wcvi_bottom.npy		      v_unrotated_wcvi_t.npy
U_3D_wcvi_left.npy		      WCVI_Grid.png
U_3D_wcvi_right.npy		      WCVI_orthogonality.png
U_3D_wcvi_top.npy		      Writing_bdy_files.ipynb


In [3]:
u_3d_left = np.load('U_3D_wcvi_left.npy'); v_3d_left_oneless = np.load('V_3D_wcvi_left.npy');

ssh_left = np.load('ssh_leftbc.npy'); 

votemper_left = np.load('votemper_leftbc.npy'); vosaline_left = np.load('vosaline_leftbc.npy');

##### Seeing the shapes of the arrays of U and V (both 2d and 3D) we see that we have one grid point less in V in the y grid)... so we make that value equal to the immediately next grid point.

In [4]:
u_2d_left = np.mean(u_3d_left, axis= 1); v_2d_left_oneless = np.mean(v_3d_left_oneless, axis=1);

v_2d_left = np.zeros(u_2d_left.shape); 
v_2d_left[:,:-1,:] = v_2d_left_oneless;
v_2d_left[:,-1,:] = v_2d_left[:,-2,:];

v_3d_left = np.zeros(u_3d_left.shape);
v_3d_left[:,:,:-1,:] = v_3d_left_oneless;
v_3d_left[:,:,-1,:] = v_3d_left[:,:,-2,:];


In [5]:
v_2d_left.shape

(43, 100, 4)

In [6]:
u_2d_left.shape

(43, 100, 4)

#### Now we will write these down in a file to make the leftmost boundary work with unstructured open boundaries

In [7]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/2d_m01.nc', 'w', zlib=True);

bdy_file.createDimension('xb', u_3d_left.shape[3]);
bdy_file.createDimension('yb', u_3d_left.shape[2]);
bdy_file.createDimension('time_counter', None);

xb = bdy_file.createVariable('xb', 'int32', ('xb',), zlib=True);
xb.units = 'indices';
xb.longname = 'x indices along left boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of y indices across all of left boundary';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

sossheig = bdy_file.createVariable('sossheig', 'float32', ('time_counter', 'yb', 'xb'), zlib=True);
vobtcrtx = bdy_file.createVariable('vobtcrtx', 'float32', ('time_counter', 'yb', 'xb'), zlib=True);
vobtcrty = bdy_file.createVariable('vobtcrty', 'float32', ('time_counter', 'yb', 'xb'), zlib=True);


sossheig[:] = ssh_left[:];
vobtcrtx[:] = u_2d_left[:];
vobtcrty[:] = v_2d_left[:];

bdy_file.close()


In [8]:
v_2d_left.shape

(43, 100, 4)

In [9]:
u_3d_left.shape

(43, 50, 100, 4)

In [10]:
v_3d_left.shape

(43, 50, 100, 4)

In [11]:
votemper_left.shape

(43, 50, 100, 4)

In [12]:
!ncdump -h /ocean/ssahu/CANYONS/bdy_files/2d_left.nc

*** ncdump: ncopen failed on /ocean/ssahu/CANYONS/bdy_files/2d_left.nc


#### NEMO has a provision to give the total velcities as input by specifying 

     ln_full_vel = .false.		        ! =T => full velocities in 3D boundary conditions
                                        ! =F => baroclinic velocities in 3D boundary conditions
                                        
         So after getting the barotropic (2D) it is not necessary to subtract the depth averaged velocities from their 3D counterparts to get the barocloinic velocities.

In [13]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_m01.nc', 'w', zlib=True);


bdy_file.createDimension('xb', u_3d_left.shape[3]);
bdy_file.createDimension('yb', u_3d_left.shape[2]);
bdy_file.createDimension('deptht', u_3d_left.shape[1]);
bdy_file.createDimension('time_counter', None);


xb = bdy_file.createVariable('xb', 'int32', ('xb',), zlib=True);
xb.units = 'indices';
xb.longname = 'x indices along left boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of y indices across all of left boundary';

deptht = bdy_file.createVariable('deptht', 'int32', ('deptht',), zlib=True);
deptht.units = 'm';
deptht.longname = 'Vertical T Levels';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

vozocrtx = bdy_file.createVariable('vozocrtx', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
vomecrty = bdy_file.createVariable('vomecrty', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
votemper = bdy_file.createVariable('votemper', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
vosaline = bdy_file.createVariable('vosaline', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);

vozocrtx[:] = u_3d_left[:];
vomecrty[:] = v_3d_left[:];
votemper[:] = votemper_left[:];
vosaline[:] = vosaline_left[:];

bdy_file.close()




#### Make the east boundary

In [14]:
u_3d_right = np.load('U_3D_wcvi_right.npy'); v_3d_right_oneless = np.load('V_3D_wcvi_right.npy');

ssh_right = np.load('ssh_rightbc.npy'); 

votemper_right = np.load('votemper_rightbc.npy'); vosaline_right = np.load('vosaline_rightbc.npy');

u_2d_right = np.mean(u_3d_right, axis= 1); v_2d_right_oneless = np.mean(v_3d_right_oneless, axis=1);

v_2d_right = np.zeros(u_2d_right.shape); 
v_2d_right[:,:-1,:] = v_2d_right_oneless;
v_2d_right[:,-1,:] = v_2d_right[:,-2,:];

v_3d_right = np.zeros(u_3d_right.shape);
v_3d_right[:,:,:-1,:] = v_3d_right_oneless;
v_3d_right[:,:,-1,:] = v_3d_right[:,:,-2,:];

In [15]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/2d_m03.nc', 'w', zlib=True);

bdy_file.createDimension('xb', u_3d_right.shape[3]);
bdy_file.createDimension('yb', u_3d_right.shape[2]);
bdy_file.createDimension('time_counter', None);

xb = bdy_file.createVariable('xb', 'int32', ('xb',), zlib=True);
xb.units = 'indices';
xb.longname = 'x indices along left boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of y indices across all of right boundary';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

sossheig = bdy_file.createVariable('sossheig', 'float32', ('time_counter', 'yb', 'xb'), zlib=True);
vobtcrtx = bdy_file.createVariable('vobtcrtx', 'float32', ('time_counter', 'yb', 'xb'), zlib=True);
vobtcrty = bdy_file.createVariable('vobtcrty', 'float32', ('time_counter', 'yb', 'xb'), zlib=True);


sossheig[:] = ssh_right[:];
vobtcrtx[:] = u_2d_right[:];
vobtcrty[:] = v_2d_right[:];

bdy_file.close()


In [16]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_m03.nc', 'w', zlib=True);


bdy_file.createDimension('xb', u_3d_right.shape[3]);
bdy_file.createDimension('yb', u_3d_right.shape[2]);
bdy_file.createDimension('deptht', u_3d_right.shape[1]);
bdy_file.createDimension('time_counter', None);


xb = bdy_file.createVariable('xb', 'int32', ('xb',), zlib=True);
xb.units = 'indices';
xb.longname = 'x indices along right boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of y indices across all of right boundary';

deptht = bdy_file.createVariable('deptht', 'int32', ('deptht',), zlib=True);
deptht.units = 'm';
deptht.longname = 'Vertical T Levels';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

vozocrtx = bdy_file.createVariable('vozocrtx', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
vomecrty = bdy_file.createVariable('vomecrty', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
votemper = bdy_file.createVariable('votemper', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
vosaline = bdy_file.createVariable('vosaline', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);

vozocrtx[:] = u_3d_right[:];
vomecrty[:] = v_3d_right[:];
votemper[:] = votemper_right[:];
vosaline[:] = vosaline_right[:];

bdy_file.close()



#### Making the north boundary

In [17]:
u_3d_top_twoless = np.load('U_3D_wcvi_top.npy'); v_3d_top_oneless = np.load('V_3D_wcvi_top.npy');

ssh_top = np.load('ssh_topbc.npy'); 

votemper_top = np.load('votemper_topbc.npy'); vosaline_top = np.load('vosaline_topbc.npy');

u_2d_top_twoless = np.mean(u_3d_top_twoless, axis= 1); v_2d_top_oneless = np.mean(v_3d_top_oneless, axis=1);

u_2d_top = np.zeros((43,4,70)); 
u_2d_top[:,:,:-2] = u_2d_top_twoless;
u_2d_top[:,:,-1] = u_2d_top[:,:,-3];
u_2d_top[:,:,-2] = u_2d_top[:,:,-3];

u_3d_top = np.zeros((43,50,4,70)); 
u_3d_top[:,:,:,:-2] = u_3d_top_twoless;
u_3d_top[:,:,:,-1] = u_3d_top[:,:,:,-3];
u_3d_top[:,:,:,-2] = u_3d_top[:,:,:,-3];


v_2d_top = np.zeros((43,4,70)); 
v_2d_top[:,:,:-1] = v_2d_top_oneless;
v_2d_top[:,:,-1] = v_2d_top[:,:,-2];


v_3d_top = np.zeros((43,50,4,70)); 
v_3d_top[:,:,:,:-1] = v_3d_top_oneless;
v_3d_top[:,:,:,-1] = v_3d_top[:,:,:,-2];

In [18]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/2d_m02.nc', 'w', zlib=True);

bdy_file.createDimension('xb', u_3d_top.shape[3]);
bdy_file.createDimension('yb', u_3d_top.shape[2]);
bdy_file.createDimension('time_counter', None);

xb = bdy_file.createVariable('xb', 'int32', ('xb',), zlib=True);
xb.units = 'indices';
xb.longname = 'x indices along north boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of y indices across all of north boundary';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

sossheig = bdy_file.createVariable('sossheig', 'float32', ('time_counter', 'yb', 'xb'), zlib=True);
vobtcrtx = bdy_file.createVariable('vobtcrtx', 'float32', ('time_counter', 'yb', 'xb'), zlib=True);
vobtcrty = bdy_file.createVariable('vobtcrty', 'float32', ('time_counter', 'yb', 'xb'), zlib=True);


sossheig[:] = ssh_top[:];
vobtcrtx[:] = u_2d_top[:];
vobtcrty[:] = v_2d_top[:];

bdy_file.close()


In [19]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_m02.nc', 'w', zlib=True);


bdy_file.createDimension('xb', u_3d_top.shape[3]);
bdy_file.createDimension('yb', u_3d_top.shape[2]);
bdy_file.createDimension('deptht', u_3d_top.shape[1]);
bdy_file.createDimension('time_counter', None);


xb = bdy_file.createVariable('xb', 'int32', ('xb',), zlib=True);
xb.units = 'indices';
xb.longname = 'x indices along north boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of y indices across all of north boundary';

deptht = bdy_file.createVariable('deptht', 'int32', ('deptht',), zlib=True);
deptht.units = 'm';
deptht.longname = 'Vertical T Levels';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

vozocrtx = bdy_file.createVariable('vozocrtx', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
vomecrty = bdy_file.createVariable('vomecrty', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
votemper = bdy_file.createVariable('votemper', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
vosaline = bdy_file.createVariable('vosaline', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);

vozocrtx[:] = u_3d_top[:];
vomecrty[:] = v_3d_top[:];
votemper[:] = votemper_top[:];
vosaline[:] = vosaline_top[:];

bdy_file.close()


In [20]:
u_2d_top.shape

(43, 4, 70)

In [21]:
v_3d_top.shape

(43, 50, 4, 70)

In [22]:
u_3d_top.shape

(43, 50, 4, 70)

In [23]:
ssh_top.shape

(43, 4, 70)

##### Making the bottom boundary

In [24]:
u_3d_bottom_twoless = np.load('U_3D_wcvi_bottom.npy'); v_3d_bottom_oneless = np.load('V_3D_wcvi_bottom.npy');

ssh_bottom = np.load('ssh_bottombc.npy'); 

votemper_bottom = np.load('votemper_bottombc.npy'); vosaline_bottom = np.load('vosaline_bottombc.npy');


u_2d_bottom_twoless = np.mean(u_3d_bottom_twoless, axis= 1); v_2d_bottom_oneless = np.mean(v_3d_bottom_oneless, axis=1);

u_2d_bottom = np.zeros((43,4,70)); 
u_2d_bottom[:,:,:-2] = u_2d_bottom_twoless;
u_2d_bottom[:,:,-1] = u_2d_bottom[:,:,-3];
u_2d_bottom[:,:,-2] = u_2d_bottom[:,:,-3];

u_3d_bottom = np.zeros((43,50,4,70)); 
u_3d_bottom[:,:,:,:-2] = u_3d_bottom_twoless;
u_3d_bottom[:,:,:,-1] = u_3d_bottom[:,:,:,-3];
u_3d_bottom[:,:,:,-2] = u_3d_bottom[:,:,:,-3];


v_2d_bottom = np.zeros((43,4,70)); 
v_2d_bottom[:,:,:-1] = v_2d_bottom_oneless;
v_2d_bottom[:,:,-1] = v_2d_bottom[:,:,-2];


v_3d_bottom = np.zeros((43,50,4,70)); 
v_3d_bottom[:,:,:,:-1] = v_3d_bottom_oneless;
v_3d_bottom[:,:,:,-1] = v_3d_bottom[:,:,:,-2];


In [25]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/2d_m04.nc', 'w', zlib=True);

bdy_file.createDimension('xb', u_3d_bottom.shape[3]);
bdy_file.createDimension('yb', u_3d_bottom.shape[2]);
bdy_file.createDimension('time_counter', None);

xb = bdy_file.createVariable('xb', 'int32', ('xb',), zlib=True);
xb.units = 'indices';
xb.longname = 'x indices along south boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of y indices across all of south boundary';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

sossheig = bdy_file.createVariable('sossheig', 'float32', ('time_counter', 'yb', 'xb'), zlib=True);
vobtcrtx = bdy_file.createVariable('vobtcrtx', 'float32', ('time_counter', 'yb', 'xb'), zlib=True);
vobtcrty = bdy_file.createVariable('vobtcrty', 'float32', ('time_counter', 'yb', 'xb'), zlib=True);


sossheig[:] = ssh_bottom[:];
vobtcrtx[:] = u_2d_bottom[:];
vobtcrty[:] = v_2d_bottom[:];

bdy_file.close()

In [26]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_m04.nc', 'w', zlib=True);


bdy_file.createDimension('xb', u_3d_bottom.shape[3]);
bdy_file.createDimension('yb', u_3d_bottom.shape[2]);
bdy_file.createDimension('deptht', u_3d_bottom.shape[1]);
bdy_file.createDimension('time_counter', None);


xb = bdy_file.createVariable('xb', 'int32', ('xb',), zlib=True);
xb.units = 'indices';
xb.longname = 'x indices along south boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of y indices across all of south boundary';

deptht = bdy_file.createVariable('deptht', 'int32', ('deptht',), zlib=True);
deptht.units = 'm';
deptht.longname = 'Vertical T Levels';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

vozocrtx = bdy_file.createVariable('vozocrtx', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
vomecrty = bdy_file.createVariable('vomecrty', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
votemper = bdy_file.createVariable('votemper', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
vosaline = bdy_file.createVariable('vosaline', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);

vozocrtx[:] = u_3d_bottom[:];
vomecrty[:] = v_3d_bottom[:];
votemper[:] = votemper_bottom[:];
vosaline[:] = vosaline_bottom[:];

bdy_file.close()

In [27]:
u_3d_bottom.shape

(43, 50, 4, 70)

In [28]:
v_3d_bottom.shape

(43, 50, 4, 70)

In [29]:
votemper_bottom.shape

(43, 50, 4, 70)